## Problem: Implement Mixed Precision Training Using `torch.cuda.amp`

### Problem Statement
Mixed precision training uses both 16-bit and 32-bit floating-point types to accelerate training and reduce memory usage without significantly affecting model performance. Your task is to implement mixed precision training for a deep learning model using PyTorch's `torch.cuda.amp`.

### Requirements

1. **Enable Mixed Precision Training**:
   - Context manager to enable mixed precision for the forward pass.
   - Scale gradients during backpropagation and ensure stability.

In [5]:
# Implement mixed precision training in PyTorch using torch.cuda.amp
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [6]:
# Define a simple model
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(10, 1)

    def forward(self, x):
        return self.fc(x)

# Generate synthetic data
X = torch.randn(1000, 10)
y = torch.randn(1000, 1)
dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Initialize model, loss function, and optimizer
model = SimpleModel().cuda()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Enable mixed precision training
scaler = torch.cuda.amp.GradScaler()

In [7]:
# Training loop
epochs = 5
for epoch in range(epochs):
    for inputs, labels in dataloader:
        inputs, labels = inputs.cuda(), labels.cuda()

        # Forward pass under autocast
        with torch.cuda.amp.autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        # Backward pass with scaled gradients
        optimizer.zero_grad()
        # TODO: Set scaler
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item():.4f}")

# Test the model on new data
X_test = torch.randn(5, 10).cuda()
with torch.no_grad(), torch.cuda.amp.autocast():
    predictions = model(X_test)
    print("Predictions:", predictions)


Epoch 1/5, Loss: 0.7239
Epoch 2/5, Loss: 1.0068
Epoch 3/5, Loss: 0.6368
Epoch 4/5, Loss: 0.4223
Epoch 5/5, Loss: 2.1796
Predictions: tensor([[ 0.0476],
        [ 0.4092],
        [ 0.1250],
        [-0.5103],
        [ 0.4922]], device='cuda:0', dtype=torch.float16)


In [8]:
# Test the model on new data
X_test = torch.randn(5, 10).cuda()
with torch.no_grad(), torch.cuda.amp.autocast():
    predictions = model(X_test)
    print("Predictions:", predictions)

Predictions: tensor([[ 0.1688],
        [ 0.1964],
        [ 0.0887],
        [-0.1493],
        [-0.1494]], device='cuda:0', dtype=torch.float16)
